In [1]:
import pandas as pd
import numpy as np

In [2]:
Parkhäuser = pd.read_csv("../Final_Data/Cleaned/1000014cleaned.csv")
Flugbewegungen = pd.read_csv("../Final_Data/Cleaned/Airport_traffic_hourly_cleaned.csv")
Gasverbrauch = pd.read_csv("../Final_Data/Cleaned/Gasverbrauch_cleaned.csv")
#Schallmessungen = pd.read_csv("../Final_Data/Cleaned/Schallmessungen_cleaned.csv", sep=";") -> zu viele Datenpunkte fehlen
Stromverbrauch = pd.read_csv("../Final_Data/Cleaned/Stromverbrauch_Basel_hourly_cleaned.csv")
Wetterdaten = pd.read_csv("../Final_Data/Cleaned/Wetterdaten_Basel_cleaned22_24.csv")
Verkehrsdaten = pd.read_csv("../Final_Data/Cleaned/1000006cleaned.csv")

In [3]:

Parkhäuser.rename(columns={"RoundedTime": "Datum"}, inplace=True)
Parkhäuser.head()
#Flugbewegungen.drop(columns=["Unnamed: 0"], inplace=True)
Flugbewegungen.head()
#Gasverbrauch.head()
#Schallmessungen.drop(columns=["Unnamed: 0"], inplace=True)
#Schallmessungen.head()
#Stromverbrauch.drop(columns=["Unnamed: 0"], inplace=True)
#Stromverbrauch.head()
#Wetterdaten.head()
#Verkehrsdaten.rename(columns={"DateTimeFrom": "Datum"}, inplace=True)
#Verkehrsdaten.head()




,Unnamed: 0,Datum,Hour,Traffic,Weekday
0,0,2022-01-01 00:00:00+00:00,0,1.0,Saturday
1,1,2022-01-01 00:00:00+00:00,1,0.0,Saturday
2,2,2022-01-01 00:00:00+00:00,2,0.0,Saturday
3,3,2022-01-01 00:00:00+00:00,3,0.0,Saturday
4,4,2022-01-01 00:00:00+00:00,4,0.0,Saturday


In [4]:
Parkhäuser.head()

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,Claramattechange,Elisabethenchange,Steinenchange,Kunstmuseumchange,Messechange,Anfoschange,Centralbahnparkingchange
0,2022-01-01 00:00:00+00:00,-7.0,-6.0,-15.0,-19.0,0.0,-2.0,-2.0,-3.0,-1.0,-23.0,-5.0,-10.0,-4.0,-7.0,0.0,-2.0
1,2022-01-01 01:00:00+00:00,-7.0,-1.0,-4.0,-9.0,-1.0,0.0,0.0,0.0,0.0,0.0,-9.0,-19.0,0.0,-8.0,-1.0,-1.0
2,2022-01-01 02:00:00+00:00,-4.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,-12.0,-23.0,0.0,-8.0,0.0,0.0
3,2022-01-01 03:00:00+00:00,-3.0,0.0,-1.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,-15.0,-10.0,0.0,-2.0,0.0,0.0
4,2022-01-01 04:00:00+00:00,-2.0,0.0,2.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,-1.0,-7.0,0.0,-2.0,0.0,1.0


In [5]:
Flugbewegungen.head()
Verkehrsdaten.head()
print(type(Verkehrsdaten))

<class 'pandas.core.frame.DataFrame'>


In [6]:
Verkehrsdaten = Verkehrsdaten.rename(columns={"DateTimeFrom": "Datum"})
print(Verkehrsdaten['Datum'].isna().sum())


0


In [7]:
#standardise the data
from sklearn.preprocessing import StandardScaler
import numpy as np
import pickle

dfs = [Parkhäuser, Flugbewegungen, Gasverbrauch, Stromverbrauch, Wetterdaten, Verkehrsdaten]

# Dictionary to store the mean and standard deviation for each dataframe
scalers = {}

# Loop through each dataframe
for i, df in enumerate(dfs):
    numeric_cols = df.select_dtypes(include=[np.number])
    if not numeric_cols.empty:
        if i == 1:  # Exclude 'Hour' column for Flugbewegungen
            numeric_cols = numeric_cols.drop(columns=['Hour'])
        print(f"Processing df{i} with columns: {numeric_cols.columns}")
        scaler = StandardScaler()
        # Fit the scaler on the dataframe and transform the data
        scaled_data = scaler.fit_transform(numeric_cols)
        
        # Save the mean and standard deviation
        scalers[f'df{i}'] = {'mean': scaler.mean_, 'scale': scaler.scale_}
        
        # Replace the original data with the standardized data
        df[numeric_cols.columns] = scaled_data
        dfs[i] = df
    else:
        print(f"df{i} has no numeric columns or is empty.")

# Assign the transformed dataframes back to their original names
Parkhäuser, Flugbewegungen, Gasverbrauch, Stromverbrauch, Wetterdaten, Verkehrsdaten = dfs

# Save the scalers dictionary for later use
with open('scalers.pkl', 'wb') as f:
    pickle.dump(scalers, f)


Processing df0 with columns: Index(['Rebgassechange', 'Clarahuuschange', 'Citychange', 'Storchenchange',
       'Post Baselchange', 'Aeschenchange', 'Bahnhof Südchange',
       'Bad. Bahnhofchange', 'Europechange', 'Claramattechange',
       'Elisabethenchange', 'Steinenchange', 'Kunstmuseumchange',
       'Messechange', 'Anfoschange', 'Centralbahnparkingchange'],
      dtype='object')
Processing df1 with columns: Index(['Unnamed: 0', 'Traffic'], dtype='object')
Processing df2 with columns: Index(['Gasverbrauch'], dtype='object')
Processing df3 with columns: Index(['Stromverbrauch'], dtype='object')
Processing df4 with columns: Index(['Unnamed: 0', 'Basel Temperature [2 m elevation corrected]',
       'Basel Precipitation Total', 'Basel Wind Speed [10 m]',
       'Basel Wind Direction [10 m]'],
      dtype='object')
Processing df5 with columns: Index(['350n_sumPW', '350n_sumLief', '350n_sumLW', '350v_sumPW',
       '350v_sumLief', '350v_sumLW', '352n_sumPW', '352n_sumLief',
       '352

In [8]:
print(type(Verkehrsdaten))
Wetterdaten.drop(columns=["Unnamed: 0"], inplace=True)
Flugbewegungen.drop(columns=["Unnamed: 0"], inplace=True)
Gasverbrauch.sort_values(by=["Datum"], inplace=True)

<class 'pandas.core.frame.DataFrame'>


In [9]:
Verkehrsdaten.head()


,Datum,350n_sumPW,350n_sumLief,350n_sumLW,350v_sumPW,350v_sumLief,350v_sumLW,352n_sumPW,352n_sumLief,352n_sumLW,...,84111104n_sumLW,84111104v_sumPW,84111104v_sumLief,84111104v_sumLW,84111108n_sumPW,84111108n_sumLief,84111108n_sumLW,84111108v_sumPW,84111108v_sumLief,84111108v_sumLW
0,2021-12-31 23:00:00+00:00,-0.429021,-0.940608,-0.796961,-0.888120,-0.953473,-0.810167,0.046689,-0.602995,-0.450842,...,-0.085348,-1.36395,1.323365,-0.331206,-0.892472,-0.571421,-1.055084,-1.120734,-0.654501,-0.611608
1,2022-01-01 00:00:00+00:00,-0.548987,-0.970116,-0.631220,-0.731198,-0.774723,-0.732415,-0.145855,-0.648232,-1.020804,...,-0.085348,-1.36395,1.425346,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471
2,2022-01-01 01:00:00+00:00,-0.867158,-0.911100,-0.796961,-0.935672,-0.794584,-0.810167,-0.338398,-0.783942,-1.020804,...,-0.284046,-1.36395,0.745474,-0.744920,-0.955306,-0.781539,-1.055084,-1.048762,-0.773270,-1.072471
3,2022-01-01 02:00:00+00:00,-1.117523,-0.970116,-0.796961,-1.049798,-1.032917,-0.732415,-0.675350,-0.693469,-1.115797,...,-0.681442,-1.36395,0.575505,-0.744920,-0.937353,-0.571421,-1.055084,-1.057758,-0.416962,-1.072471
4,2022-01-01 03:00:00+00:00,-1.185330,-0.881591,-0.796961,-1.344621,-1.052778,-0.810167,-0.883939,-0.874416,-1.115797,...,-0.482744,-1.36395,0.269563,-0.744920,-1.045068,-0.571421,-1.055084,-1.201702,-0.416962,-1.072471


In [ ]:
Parkhäuser['Datum'] = pd.to_datetime(Parkhäuser['Datum'], errors='coerce')
Wetterdaten['Datum'] = pd.to_datetime(Wetterdaten['Datum'], errors='coerce')
Flugbewegungen['Datum'] = pd.to_datetime(Flugbewegungen['Datum'], errors='coerce')
Gasverbrauch['Datum'] = pd.to_datetime(Gasverbrauch['Datum'], errors='coerce')
Stromverbrauch['Datum'] = pd.to_datetime(Stromverbrauch['Datum'], errors='coerce')
Verkehrsdaten['Datum'] = pd.to_datetime(Verkehrsdaten['Datum'], errors='coerce')



# Convert timezone but keep time by explicitly formatting the datetime
Parkhäuser['Datum'] = Parkhäuser['Datum'].dt.tz_convert(None)
Wetterdaten['Datum'] = Wetterdaten['Datum'].dt.tz_convert(None)
Flugbewegungen['Datum'] = Flugbewegungen['Datum'].dt.tz_convert(None)
Gasverbrauch['Datum'] = Gasverbrauch['Datum'].dt.tz_convert(None)
Stromverbrauch['Datum'] = Stromverbrauch['Datum'].dt.tz_convert(None)
Verkehrsdaten['Datum'] = Verkehrsdaten['Datum'].dt.tz_convert(None)

                                                                                                # The merge isnt correct because it erases the Hour format and therefore setting the entire Day to one value?!
# Merge all dataframes on the 'Datum' column
final_df = Parkhäuser.merge(Wetterdaten, on='Datum', how='inner') \
                      .merge(Flugbewegungen, on='Datum', how='inner') \
                      .merge(Stromverbrauch, on='Datum', how='inner') \
                      .merge(Verkehrsdaten, on='Datum', how='inner') \
                      .merge(Gasverbrauch, on='Datum', how='inner') \
#                      .merge(Schallmessungen, on='Datum', how='inner') \ -> too many NaN values
# Display the first few rows of the merged dataframe
final_df.head(20)

TypeError: Cannot convert tz-naive timestamps, use tz_localize to localize

In [34]:
final_df.head(25)

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,...,84111104v_sumPW,84111104v_sumLief,84111104v_sumLW,84111108n_sumPW,84111108n_sumLief,84111108n_sumLW,84111108v_sumPW,84111108v_sumLief,84111108v_sumLW,Gasverbrauch
0,2022-01-01 00:00:00,-0.489850,-1.162123,-0.163777,-1.110018,-0.000078,-0.258065,-0.245463,-0.158260,-0.151263,...,-1.36395,1.425346,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494
1,2022-01-01 01:00:00,-0.489850,-1.162123,-0.163777,-1.110018,-0.000078,-0.258065,-0.245463,-0.158260,-0.151263,...,-1.36395,1.425346,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494
2,2022-01-01 02:00:00,-0.489850,-1.162123,-0.163777,-1.110018,-0.000078,-0.258065,-0.245463,-0.158260,-0.151263,...,-1.36395,1.425346,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494
3,2022-01-01 03:00:00,-0.489850,-1.162123,-0.163777,-1.110018,-0.000078,-0.258065,-0.245463,-0.158260,-0.151263,...,-1.36395,1.425346,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494
4,2022-01-01 04:00:00,-0.489850,-1.162123,-0.163777,-1.110018,-0.000078,-0.258065,-0.245463,-0.158260,-0.151263,...,-1.36395,1.425346,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494
5,2022-01-01 05:00:00,-0.489850,-1.162123,-0.163777,-1.110018,-0.000078,-0.258065,-0.245463,-0.158260,-0.151263,...,-1.36395,1.425346,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494
6,2022-01-01 06:00:00,-0.489850,-1.162123,-0.163777,-1.110018,-0.000078,-0.258065,-0.245463,-0.158260,-0.151263,...,-1.36395,1.425346,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494
7,2022-01-01 07:00:00,-0.489850,-1.162123,-0.163777,-1.110018,-0.000078,-0.258065,-0.245463,-0.158260,-0.151263,...,-1.36395,1.425346,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494
8,2022-01-01 08:00:00,-0.489850,-1.162123,-0.163777,-1.110018,-0.000078,-0.258065,-0.245463,-0.158260,-0.151263,...,-1.36395,1.425346,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494
9,2022-01-01 09:00:00,-0.489850,-1.162123,-0.163777,-1.110018,-0.000078,-0.258065,-0.245463,-0.158260,-0.151263,...,-1.36395,1.425346,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494


In [30]:
test_dataFrame = [Parkhäuser,
                Wetterdaten,
                Flugbewegungen,
                Gasverbrauch,
                Stromverbrauch,
                Verkehrsdaten]

for i in test_dataFrame:
    print(i['Datum'].max())

2024-10-21 13:00:00
2024-11-22 23:00:00
2024-11-22 00:00:00
2024-11-24 04:00:00
2024-11-20 22:00:00
2024-11-19 22:00:00


In [32]:
Parkhäuser[Parkhäuser["Datum"] == "2024-10-21 13:00:00"]

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,Claramattechange,Elisabethenchange,Steinenchange,Kunstmuseumchange,Messechange,Anfoschange,Centralbahnparkingchange
24589,2024-10-21 13:00:00,0.000102,-0.000213,-0.000368,-0.000099,-0.000078,-0.000173,-0.000199,-0.000464,-0.000215,-0.000155,-0.000137,-0.000189,-0.000277,-0.000221,-0.000242,-0.000254


In [12]:

final_df['Datum'] = final_df.apply(lambda row: row['Datum'].replace(hour=row['Hour']), axis=1)

# Drop the Hour column if no longer needed
final_df = final_df.drop('Hour', axis=1)

# Display the updated DataFrame
print(final_df.head())

                Datum  Rebgassechange  Clarahuuschange  Citychange  \
0 2022-01-01 00:00:00        -0.48985        -1.162123   -0.163777   
1 2022-01-01 01:00:00        -0.48985        -1.162123   -0.163777   
2 2022-01-01 02:00:00        -0.48985        -1.162123   -0.163777   
3 2022-01-01 03:00:00        -0.48985        -1.162123   -0.163777   
4 2022-01-01 04:00:00        -0.48985        -1.162123   -0.163777   

   Storchenchange  Post Baselchange  Aeschenchange  Bahnhof Südchange  \
0       -1.110018         -0.000078      -0.258065          -0.245463   
1       -1.110018         -0.000078      -0.258065          -0.245463   
2       -1.110018         -0.000078      -0.258065          -0.245463   
3       -1.110018         -0.000078      -0.258065          -0.245463   
4       -1.110018         -0.000078      -0.258065          -0.245463   

   Bad. Bahnhofchange  Europechange  ...  84111104v_sumPW  84111104v_sumLief  \
0            -0.15826     -0.151263  ...         -1.36395   

# Check for rows with NaN values
nan_rows = final_df[final_df.isnull().any(axis=1)]

# Display the rows with NaN values
nan_rows.head()

In [24]:
type(final_df[["Datum"]])

pandas.core.frame.DataFrame

In [13]:
final_df.to_csv("../Working_DataFrame.csv", index=False)

In [28]:

#final_df.tail(40)
final_df['Datum'].dtype
import numpy as np

cutoff = np.datetime64('2024-11-20 10:00:00')
final_df = final_df[final_df['Datum'] <= cutoff]
final_df.tail(40)

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,...,84111104v_sumPW,84111104v_sumLief,84111104v_sumLW,84111108n_sumPW,84111108n_sumLief,84111108n_sumLW,84111108v_sumPW,84111108v_sumLief,84111108v_sumLW,Gasverbrauch
24560,2024-10-20 08:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537
24561,2024-10-20 09:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537
24562,2024-10-20 10:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537
24563,2024-10-20 11:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537
24564,2024-10-20 12:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537
24565,2024-10-20 13:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537
24566,2024-10-20 14:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537
24567,2024-10-20 15:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537
24568,2024-10-20 16:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537
24569,2024-10-20 17:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537


In [14]:
#2 NaN in Stromverbrauch 2022-10-30 01:00:00 & 2023-10-29 01:00:00

# Verkehrsdatenset hat 5 NaN values 2024-10-20 22:00:00 & 2024-10-20 23:00:00 & 2024-10-21 00:00:00 & 2024-10-21 01:00:00 & 2024-10-21 02:00:00	

# Wetter hat 5 NaN values 2023-12-22 00:00:00 & 2023-12-22 01:00:00 & 2023-12-22 02:00:00 & 2023-12-22 03:00:00 & 2023-12-22 04:00:00
# habe nochmal nachgeschaut und es ist der ganze 22 und 23 Dezember 2023 || ist aber nur eine Auflösung die ich ausgewählt habe, sollte mit der anderen Auflösung nicht mehr vorkommen
